In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
sys.path.append('../')
import torch
import numpy as np
import argparse
import matplotlib.pyplot as plt
from model import AlignmentModel
from torch.utils.data import DataLoader
from vqa_dataset import VQADataset
from cc_dataset import CCDataset

In [4]:
# Split CC datasets into a subset - 50%
device = "cuda:1"

# Load Model
model_path = "../checkpoints/2020_12_23_1/epch0_bidx6400.pt"
model = AlignmentModel().to(device)
model_data = torch.load(model_path)["model_state_dict"]
model_data.pop("text_encoder.model.embeddings.position_ids")

tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
         140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
         154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
         168, 169, 170, 171, 172, 173, 174, 175, 176

In [5]:
def collate_fn(batch):
    image_feats = []
    text_feats = []
    image_ids = []
    captions = []
    for sample in batch:
        image_feats.append(sample["img_feat"])
        text_feats.append(sample["text_feat"])
        if "image_id" in sample:
            image_ids.append(sample["image_id"])
        if "caption" in sample:
            captions.append(sample["caption"])
    image_feats = torch.mean(torch.stack(image_feats), dim=1).to(device)
    text_feats = torch.stack(text_feats).to(device)
    return [image_feats, text_feats, image_ids, captions]

In [6]:
class Namespace:
    def __init__(self, opts):
        self.__dict__.update(opts)

In [7]:
pretrain_args = argparse.Namespace(
    max_seq_len = 16,
    annotation_path = '/private/home/sash/.cache/torch/mmf/data/datasets/cc/defaults/annotations/train_all.npy',
    batch_size = 32,
    feature_path = '/private/home/sash/.cache/torch/mmf/data/datasets/cc/defaults/features/cc_train.lmdb',
)

In [8]:
model.load_state_dict(model_data)
model.eval()

AlignmentModel(
  (image_encoder): ImageEncoder(
    (encoder): Sequential(
      (0): Linear(in_features=2048, out_features=2048, bias=True)
      (1): ReLU()
      (2): Linear(in_features=2048, out_features=2048, bias=True)
    )
  )
  (text_encoder): TextEncoder(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_fe

In [43]:
import numpy as np

dataset = CCDataset(pretrain_args)
pretrain_dataloader = DataLoader(
    dataset,
    batch_size=pretrain_args.batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    drop_last=True,
)

passing_img = None
passing_cap = None
chunk_size = 200
with torch.no_grad():
    for batch_idx, batch in enumerate(pretrain_dataloader):
        image_feat_batch = batch[0]  # B x 2048
        text_feat_batch = batch[1] # B x 2 x 16
        image_ids = batch[2]
        captions = batch[3]
        
        image_repr, text_repr = model(image_feat_batch, text_feat_batch)
        dotproduct = image_repr.mm(text_repr.t())
        diag = dotproduct.detach().cpu().diagonal()
        mask = diag > 0
        mask_ids = np.array([idx for idx in range(len(list(mask))) if mask[idx].item()])
        passing_image_ids = np.take(np.array(image_ids), mask_ids)
        passing_captions = np.take(np.array(captions), mask_ids)
        if passing_img is None:
            passing_img = passing_image_ids
        else:
            passing_img = np.concatenate((passing_img, passing_image_ids))
        
        if passing_cap is None:
            passing_cap = passing_captions
        else:
            passing_cap = np.concatenate((passing_cap, passing_captions))
            
        if batch_idx % chunk_size == 0 and batch_idx != 0:
            print(f"{batch_idx - chunk_size}_{batch_idx} Done")
            torch.save([passing_img, passing_cap], f"../checkpoints/2020_12_23_1/passing_cc/passing_cc_50pc_{batch_idx - chunk_size}_{batch_idx}.list")
            passing_img = None
            passing_cap = None
            
torch.save(passing_image_ids, "passing_cc_50pc.list")

0_200 Done
200_400 Done
400_600 Done
600_800 Done


KeyboardInterrupt: 

In [ ]:
# 600_800 Done

In [24]:
import numpy as np
image_ids_array = np.array(image_ids)
filtered_image_ids = np.ma.array(image_ids_array, mask=np.array(mask), shrink=True)

In [25]:
filtered_image_ids

masked_array(data=['1510438788', --, '1510439783', '151044062',
                   '143205192', --, '1432055379', --, '1510441362', --,
                   --, --, '1510441777', '1510444045', '1529944185',
                   '1529944663', '1432060500', --, '151044743',
                   '1510448150', '1529944954', --, '143206376', --, --,
                   '1510451991', '1510452621', --, '1432067420',
                   '143206935', '152994627', '1529948674'],
             mask=[False,  True, False, False, False,  True, False,  True,
                   False,  True,  True,  True, False, False, False, False,
                   False,  True, False, False, False,  True, False,  True,
                    True, False, False,  True, False, False, False, False],
       fill_value='N/A',
            dtype='<U10')